In [77]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, csr_matrix
from sklearn import naive_bayes
# from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.preprocessing import StandardScaler

In [78]:
data=pd.read_csv('Hindustan_Clean.csv')

In [79]:
data.head()

,label,clean_text,word_count,prop_count,noun_count,adj_count,verb_count,stop_count
0,7,stoke to unleash promising spin duo on india a...,15,3,4,1,1,5
1,7,more than man ganguly s bonafide remark on wom...,14,1,6,1,1,4
2,7,rohit s chance to make up for wc gavaskar s bi...,15,4,3,1,1,5
3,7,watch crowd queue up outside shami s house aft...,13,3,5,0,2,2
4,7,hafeez slams australia for pakistan s warm up ...,14,5,2,2,1,3


In [80]:
data.describe()

,label,word_count,prop_count,noun_count,adj_count,verb_count,stop_count
count,214.00000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,4.28972,11.897196,2.210280,3.635514,0.971963,1.514019,2.780374
std,2.24135,2.162135,1.893326,1.570817,0.913724,0.912763,1.921802
min,1.00000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.00000,10.000000,1.000000,3.000000,0.000000,1.000000,1.000000
50%,4.00000,12.000000,2.000000,3.000000,1.000000,1.000000,3.000000
75%,6.00000,13.000000,3.000000,5.000000,1.000000,2.000000,4.000000
max,8.00000,17.000000,10.000000,9.000000,4.000000,5.000000,11.000000


In [81]:
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        def range_(value):
          if (value<lower_limit):
            return lower_limit 
          if (value>upper_limit):
            return upper_limit
          else:
            return value

        df[col]=df[col].apply(range_)
        
    return df

columns_=['prop_count','noun_count','adj_count','verb_count','stop_count']
remove_outliers(data,columns_)

,label,clean_text,word_count,prop_count,noun_count,adj_count,verb_count,stop_count
0,7,stoke to unleash promising spin duo on india a...,15,3.0,4.0,1.0,1.0,5.0
1,7,more than man ganguly s bonafide remark on wom...,14,1.0,6.0,1.0,1.0,4.0
2,7,rohit s chance to make up for wc gavaskar s bi...,15,4.0,3.0,1.0,1.0,5.0
3,7,watch crowd queue up outside shami s house aft...,13,3.0,5.0,0.0,2.0,2.0
4,7,hafeez slams australia for pakistan s warm up ...,14,5.0,2.0,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...
209,5,indonesia s prambanan temple testimony to indi...,13,6.0,2.0,0.0,0.0,2.0
210,5,mooli for weight loss and immunity 7 benefit 7...,14,0.0,7.0,1.0,1.0,3.0
211,5,we approve crispr gene edit to treat sickle ce...,10,0.0,4.0,1.0,3.0,2.0
212,5,sleep deprivation and vertigo here s how lack ...,13,0.0,5.0,1.0,2.0,5.0


In [82]:
data.describe()

,label,word_count,prop_count,noun_count,adj_count,verb_count,stop_count
count,214.00000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,4.28972,11.897196,2.135514,3.630841,0.927570,1.497664,2.768692
std,2.24135,2.162135,1.674260,1.556203,0.808196,0.867376,1.878717
min,1.00000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.00000,10.000000,1.000000,3.000000,0.000000,1.000000,1.000000
50%,4.00000,12.000000,2.000000,3.000000,1.000000,1.000000,3.000000
75%,6.00000,13.000000,3.000000,5.000000,1.000000,2.000000,4.000000
max,8.00000,17.000000,6.000000,8.000000,2.500000,3.500000,8.500000


In [83]:
# X=data.drop(columns=['label'])
Y=data['label']

In [84]:
word_tfidf=TfidfVectorizer()

In [85]:
word_tfidf.fit(data['clean_text'].values)

TfidfVectorizer()

In [86]:
word_vectors_tfidf=word_tfidf.transform(data.clean_text.values)

In [87]:
word_vectors_tfidf.shape

(214, 1266)

In [88]:
word_vectors_tfidf

<214x1266 sparse matrix of type '<class 'numpy.float64'>'
	with 2407 stored elements in Compressed Sparse Row format>

Here Note Size of the 214 * 1266

In [89]:
# your_dataframe['normalized_feature'] = np.log1p(your_dataframe['feature'])

data['adj_count']=np.log1p(data['adj_count'])
data['prop_count']=np.log1p(data['prop_count'])
data['noun_count']=np.log1p(data['noun_count'])
data['verb_count']=np.log1p(data['verb_count'])
data['stop_count']=np.log1p(data['stop_count'])





In [90]:
data.head()

,label,clean_text,word_count,prop_count,noun_count,adj_count,verb_count,stop_count
0,7,stoke to unleash promising spin duo on india a...,15,1.386294,1.609438,0.693147,0.693147,1.791759
1,7,more than man ganguly s bonafide remark on wom...,14,0.693147,1.945910,0.693147,0.693147,1.609438
2,7,rohit s chance to make up for wc gavaskar s bi...,15,1.609438,1.386294,0.693147,0.693147,1.791759
3,7,watch crowd queue up outside shami s house aft...,13,1.386294,1.791759,0.000000,1.098612,1.098612
4,7,hafeez slams australia for pakistan s warm up ...,14,1.791759,1.098612,1.098612,0.693147,1.386294


In [91]:
features = ['prop_count',
       'noun_count', 'adj_count', 'verb_count', 'stop_count']

# Meta features
feature_set1 = data[features]
feature_set1

,prop_count,noun_count,adj_count,verb_count,stop_count
0,1.386294,1.609438,0.693147,0.693147,1.791759
1,0.693147,1.945910,0.693147,0.693147,1.609438
2,1.609438,1.386294,0.693147,0.693147,1.791759
3,1.386294,1.791759,0.000000,1.098612,1.098612
4,1.791759,1.098612,1.098612,0.693147,1.386294
...,...,...,...,...,...
209,1.945910,1.098612,0.000000,0.000000,1.098612
210,0.000000,2.079442,0.693147,0.693147,1.386294
211,0.000000,1.609438,0.693147,1.386294,1.098612
212,0.000000,1.791759,0.693147,1.098612,1.791759


In [92]:
# scale=StandardScaler()
# x_new=scale.fit_transform(feature_set1)
# x_new

In [93]:
data.columns

Index(['label', 'clean_text', 'word_count', 'prop_count', 'noun_count',
       'adj_count', 'verb_count', 'stop_count'],
      dtype='object')

In [94]:

X= hstack([word_vectors_tfidf, csr_matrix(feature_set1)], "csr")
X

<214x1271 sparse matrix of type '<class 'numpy.float64'>'
	with 3318 stored elements in Compressed Sparse Row format>

Note Here Size 214*1272 that is 6 features we also added 

In [95]:
x_train,x_test,y_train,y_test=train_test_split(X , Y , test_size=0.2 , random_state=20)

In [96]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((171, 1271), (43, 1271), (171,), (43,))

In [102]:
from sklearn.ensemble import RandomForestClassifier

In [103]:
model=RandomForestClassifier(n_estimators=100)

In [104]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [105]:
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)

In [106]:
accuracy_score(y_test,test_predict)

0.6046511627906976

In [107]:
accuracy_score(y_train,train_predict)

1.0